# GPON

In [1]:
# Importación de librerias
import pandas as pd
import numpy as np
import warnings

# Función

In [41]:
# Creación de función 
def GPON (plan_detallado_path,fechas,outputh_path_mail,outputh_path_sms):

    # Ignorar advertencias de grupos de captura en regex
    warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

    #Lectura de plan detallado
    plan_detallado = pd.read_csv(plan_detallado_path)

    #Lectura de cruce
    cruce = pd.read_csv(r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Cruce\CCMKT260720211210_Gestion_cuentas_migracion 2.csv")

    #Formateo de fechas 
    plan_detallado['Fecha de ventana'] = pd.to_datetime(plan_detallado['Fecha de ventana'], format='%d/%m/%Y')

    #Especificar fechas de filtrado
    fechas = pd.to_datetime(fechas)
    fechas_str = fechas.strftime('%Y-%m-%d').to_list()
    print("SECCIÓN DE REGISTROS POR FECHAS")
    print(f"Las fechas seleccionadas para filtrar son: {fechas_str}")

    #Filtrar el plan por las fechas
    fechas_filtradas = plan_detallado[plan_detallado['Fecha de ventana'].isin(fechas)].copy()
    print(f"Registros encontrados en el plan detallado en base a las fechas seleccionadas: {len(fechas_filtradas)}")
    print()

    #Homologación de dtypes y quitar espacios en cuenta y cliente de fechas_filtradas y cruce
    fechas_filtradas['Cliente'] = fechas_filtradas['Cliente'].astype(str).str.strip()
    cruce['CUENTA'] = cruce['CUENTA'].astype(str).str.strip()

    # Cruce con archivo CC con las fechas_filtradas
    cuentas_filtradas = cruce[cruce['CUENTA'].isin(fechas_filtradas['Cliente'])].copy()
    print("SECCIÓN DE CRUCE CON CONTRO DE CAMBIOS")
    print(f"Registros encontrados en el plan detallado dentro del CCMKT260720211210_Gestion_Cuentas: {len(cuentas_filtradas)}")
    print()

    #Tomar solo los campos que deseamos para email
    cuentas_filtradas =cuentas_filtradas[['CUENTA','NOMBRE_PRINCIPAL_CLI','APE_PATERNO','EMAIL','TELEFONO_CELULAR_PRINCIPAL']] 

    #reseteo de index
    archivo_gpon = cuentas_filtradas.reset_index(drop=True)

    #renombres a columnas
    archivo_gpon.rename(columns={'NOMBRE_PRINCIPAL_CLI':'NOMBRES', 'TELEFONO_CELULAR_PRINCIPAL': 'TELEFONO'},inplace= True)

    print("SECCIÓN DE LIMPIEZA DE NOMBRES")
    # Limpiar la columna 'NOMBRES'
    if 'NOMBRES' in archivo_gpon.columns:
        archivo_gpon['NOMBRES'] = (
            archivo_gpon['NOMBRES']
            .str.strip()
            .replace(['', ' ', '  ', '   '], np.nan, regex=False)
            .replace(r'[^a-zA-Z\s]', np.nan, regex=True)
            .replace('.', '')
            .fillna('ESTIMADO')
            .str.upper()
            .str.split(" ").str.get(0)
            
        )
        # Remplazo de Marias 
        archivo_gpon['NOMBRES'] = archivo_gpon['NOMBRES'].replace('MA','MARIA')
        # Guardar en una matriz booleana aquellos nombres =< a 2 
        nombres_2_digitos = archivo_gpon['NOMBRES'].astype('str').str.len() <= 2
        # Hacer un replace con la matriz booleana usando el método .loc 
        archivo_gpon.loc[nombres_2_digitos, 'NOMBRES'] = 'ESTIMADO'
        print("Cantidad de nombres llenados con 'ESTIMADO':", (archivo_gpon['NOMBRES'] == 'ESTIMADO').sum())
    else:
        print("Error: La columna 'NOMBRES' no existe en el DataFrame.")

    
    #Declaración de sembrados mail
    # Crear un diccionario con los datos proporcionados
    sembrados_mail = {
        'CUENTA': ['MKD000001', 'MKD000002', 'MKD000003', 'MKD000004', 'MKD000005', 'MKD000006'],
        'NOMBRES': ['OILED', 'KARLA', 'LILIANA', 'ROBERTO', 'CLAUDIA', 'LEONARDO'],
        'APE_PATERNO': ['SAENZ', 'PUENTES', 'GOMEZ', 'JASSO', 'ARAUJO', 'GONZALEZ'],
        'EMAIL': ['osaenzh@gmail.com', 'puenteskarla302@gmail.com', 'lgomez@izzi.mx', 
                'roberto.jassop@gmail.com', 'caraujo@izzi.mx', 'lgonzalezc@izzi.mx']
        }
    
     # Crear el DataFrame para mail
    df_sembrados_mail = pd.DataFrame(sembrados_mail)

    #Declaración de sembrados sms
    sembrados_sms  = {
        'CUENTA': ['MKD000001', 'MKD000002', 'MKD000003', 'MKD000004', 'MKD000005', 'MKD000006'],
        'TELEFONO': ['5520786331', '5566312529', '7775638290', '5521496283', '5520071387', '5518298039']
        }

    # Crear el DataFrame para sms
    df_sembrados_sms = pd.DataFrame(sembrados_sms)

    # Concatenar df_sembrados a archivo_gpon
    result_email = pd.concat([archivo_gpon, df_sembrados_mail], axis=0)
    result_sms = pd.concat([archivo_gpon, df_sembrados_sms], axis=0)

    #Separación de archivos email y sms
    result_email = result_email[['CUENTA','NOMBRES','APE_PATERNO','EMAIL']]
    result_sms = result_sms[['CUENTA','TELEFONO']]
  
    
    #Borrar registros nulos
    result_email.dropna(subset= 'EMAIL',inplace=True)
    result_sms.dropna(subset= 'TELEFONO',inplace=True)

    #Borrar registros duplicados
    result_email = result_email.drop_duplicates()
    result_sms = result_sms.drop_duplicates()

    #Conteo final
    print(F"El dataframe GPON mail tiene: {len(result_email)} registros ")
    print(F"El dataframe GPON sms tiene: {len(result_sms)} registros ")

    #Manejo de dtypes
    result_sms['TELEFONO'] = result_sms['TELEFONO'].astype("int")

    #Exportaciones
    result_email.to_csv(outputh_path_mail, index = False, encoding = "latin1")
    result_sms.to_csv(outputh_path_sms, index = False) 


    return 'Felicidades, has creado tus archivos GPON'


In [42]:
#Uso de la función

GPON(r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Ventanas\2025\Mayo\PLAN DETALLADO DE MAYO 2025.csv",
     ["2025-05-06","2025-05-07", "2025-05-08"],
     r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Ventanas\2025\Mayo\6, 7 y 8\BD_GPON_MAIL_060708_MAY_V2.csv",
     r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Ventanas\2025\Mayo\6, 7 y 8\BD_GPON_SMS_060708_MAY_V2.csv")

SECCIÓN DE REGISTROS POR FECHAS
Las fechas seleccionadas para filtrar son: ['2025-05-06', '2025-05-07', '2025-05-08']
Registros encontrados en el plan detallado en base a las fechas seleccionadas: 79

SECCIÓN DE CRUCE CON CONTRO DE CAMBIOS
Registros encontrados en el plan detallado dentro del CCMKT260720211210_Gestion_Cuentas: 76

SECCIÓN DE LIMPIEZA DE NOMBRES
Cantidad de nombres llenados con 'ESTIMADO': 0
El dataframe GPON mail tiene: 82 registros 
El dataframe GPON sms tiene: 81 registros 


'Felicidades, has creado tus archivos GPON'

# Experimentación

In [4]:
plan_detallado = pd.read_csv(r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Ventanas\2025\Marzo\Plan detallado de marzo 2025 VS.csv")

In [5]:
plan_detallado.columns

Index(['Cliente', 'Serie', 'HUB', 'Puerto', 'Segmento', 'PLAY', 'PAQUETE',
       'Calle ', 'Numero ext', 'colonia', 'CP', 'Municipio', 'Estatus ',
       'Base oiled 15nov', 'Contador', 'Migrados', 'Nodo', 'Fecha de ventana'],
      dtype='object')

In [ ]:
plan_detallado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Cliente           331 non-null    int64 
 1   Serie             331 non-null    object
 2   HUB               331 non-null    object
 3   Puerto            331 non-null    object
 4   Segmento          331 non-null    object
 5   PLAY              331 non-null    object
 6   PAQUETE           331 non-null    object
 7   Calle             331 non-null    object
 8   Numero ext        331 non-null    object
 9   colonia           331 non-null    object
 10  CP                331 non-null    int64 
 11  Municipio         331 non-null    object
 12  Estatus           331 non-null    object
 13  Base oiled 15nov  331 non-null    int64 
 14  Contador          331 non-null    int64 
 15  Migrados          331 non-null    object
 16  Nodo              331 non-null    object
 17  Fecha de ventana

In [7]:
#Formato de fechas 
plan_detallado['Fecha de ventana'] = pd.to_datetime(plan_detallado['Fecha de ventana'], format='%d/%m/%Y')

In [8]:
plan_detallado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Cliente           331 non-null    int64         
 1   Serie             331 non-null    object        
 2   HUB               331 non-null    object        
 3   Puerto            331 non-null    object        
 4   Segmento          331 non-null    object        
 5   PLAY              331 non-null    object        
 6   PAQUETE           331 non-null    object        
 7   Calle             331 non-null    object        
 8   Numero ext        331 non-null    object        
 9   colonia           331 non-null    object        
 10  CP                331 non-null    int64         
 11  Municipio         331 non-null    object        
 12  Estatus           331 non-null    object        
 13  Base oiled 15nov  331 non-null    int64         
 14  Contador          331 non-

In [9]:
#Declaración de fechas para filtrado
fechas = pd.to_datetime(['2025-03-25','2025-03-26', '2025-03-27'])

In [10]:
#Filtro de plan detallado por las fechas que establecimos
fechas_filtradas = plan_detallado[plan_detallado['Fecha de ventana'].isin(fechas)]

In [ ]:
# Comprobar valores de fechas en fechas_filtradas 
fechas_filtradas['Fecha de ventana'].unique()

<DatetimeArray>
['2025-03-26 00:00:00', '2025-03-27 00:00:00', '2025-03-25 00:00:00']
Length: 3, dtype: datetime64[ns]

In [ ]:
#Conteo de filtro de fechas con plan detallado
len(fechas_filtradas)

98

In [ ]:
fechas_filtradas

,Cliente,Serie,HUB,Puerto,Segmento,PLAY,PAQUETE,Calle,Numero ext,colonia,CP,Municipio,Estatus,Base oiled 15nov,Contador,Migrados,Nodo,Fecha de ventana
191,27451454,IPHO00531408,GPE,GPE 1-4-4,Residencial,VIT,IZZI 60 + IZZITV HD,AV PUNTA ESMERALDA,132,RESIDENCIAL PUNTA ESMERALDA,67289,JUAREZ,Activo,27451454,1,#N/D,JUAR9N06,2025-03-26
192,23576484,IPHO00532a9d,GPE,GPE 1-4-4,Residencial,IT,IZZI 40,ZAFIRO,310,RESIDENCIAL PUNTA ESMERALDA,67289,JUAREZ,Activo,23576484,1,#N/D,JUAR9N06,2025-03-26
193,22087597,IPHO005323e9,GPE,GPE 1-4-4,Residencial,VIT,IZZI 80 XTREMO + IZZI TV HD XTREMO_R,ZAFIRO,239,RESIDENCIAL PUNTA ESMERALDA,67289,JUAREZ,Activo,22087597,1,#N/D,JUAR9N06,2025-03-26
194,23957515,IPHO00532622,GPE,GPE 1-4-4,Residencial,VIT,IZZI 80 + IZZITV HD CON VIX PREMIUM,AV PUNTA ESMERALDA,134,RESIDENCIAL PUNTA ESMERALDA,67289,JUAREZ,Activo,23957515,1,#N/D,JUAR9N06,2025-03-26
195,30484218,IPHO00532110,GPE,GPE 1-4-4,Residencial,VIT,IZZI 30_ UNLIMITED XTREMO + IZZI TV HD UNLIMIT...,ZAFIRO,270,RESIDENCIAL PUNTA ESMERALDA,67289,JUAREZ,Activo,30484218,1,#N/D,JUAR9N06,2025-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,35321255,IPHO00516e2d,GPE,GPE 1-6-2,Residencial,IT,IZZI 60,AV PLUTARCO ELIAS CALLES,1701 LAS GAVIOTAS A,FRACC NUEVA JOYA,67168,GUADALUPE,Activo,35321255,1,#N/D,GPARGPN01,2025-03-25
327,35739067,IPHO00532620,GPE,GPE 1-6-2,Residencial,VIT,IZZI 60 + IZZITV HD,AV PLUTARCO ELIAS CALLES,1701 LAS GAVIOTAS A,FRACC NUEVA JOYA,67168,GUADALUPE,Activo,35739067,1,#N/D,GPARGPN01,2025-03-25
328,35194912,IPHO005528d8,GPE,GPE 1-6-2,Residencial,VIT,IZZI 150 + IZZITV HD,AV PLUTARCO ELIAS CALLES,1701 LAS GAVIOTAS A,FRACC NUEVA JOYA,67168,GUADALUPE,Activo,35194912,1,#N/D,GPARGPN01,2025-03-25
329,35804924,IPHO00519364,GPE,GPE 1-6-2,Residencial,IT,IZZI 150,AV PLUTARCO ELIAS CALLES,1701LAS GAVIOTAS A,FRACC NUEVA JOYA,67168,GUADALUPE,Activo,35804924,1,#N/D,GPARGPN01,2025-03-25


In [14]:
#cruce
cruce = pd.read_csv(r"C:\Users\lgonzalezc\Documents\Trabajo\DB\GPON\Cruce\CCMKT260720211210_Gestion_cuentas_migracion 2.csv")

In [15]:
#Homologación de dtypes y quitar espacios 
fechas_filtradas['Cliente'] = fechas_filtradas['Cliente'].astype(str).str.strip()
cruce['CUENTA'] = cruce['CUENTA'].astype(str).str.strip()

C:\Users\lgonzalezc\AppData\Local\Temp\ipykernel_26884\87022453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fechas_filtradas['Cliente'] = fechas_filtradas['Cliente'].astype(str).str.strip()


In [16]:
cruce.columns

Index(['ORIG_ID', 'EMPRESA', 'DIVISION', 'REGION', 'HUB', 'PLAZA', 'ID_TRANS',
       'NODO', 'RAMA', 'CUENTA', 'CALLE', 'NUMERO_INTERIOR', 'NUMERO_EXTERIOR',
       'ENTRE_CALLES', 'MANZANA', 'LOTE', 'COLONIA', 'CP',
       'MUNICIPIO_DELEGACION', 'CIUDAD', 'ESTADO',
       'TELEFONO_CELULAR_PRINCIPAL', 'TELEFONO_FIJO_PRINCIPAL', 'EMAIL',
       'NOMBRE_PRINCIPAL_CLI', 'APE_PATERNO', 'APE_MATERNO', 'PLAY', 'PAQUETE',
       'SEGMENTO', 'FACTURACION', 'FECHA_INSTALACION',
       'PLATAFORMA_TELCO_LEGACY', 'TIPO_ONT', 'MODELO_CAJA_VIDEO_CLI',
       'MODELO_EQ_INTERNET', 'SIPRE_ID', 'FLG_HFC', 'FLG_GPON', 'FLG_FTTH_MTY',
       'FLG_FTTH_AXTEL', 'TIPO_TECNOLOGIA', 'TIPO', 'LONGITUD', 'LATITUD',
       'REG'],
      dtype='object')

In [17]:
cruce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827571 entries, 0 to 827570
Data columns (total 46 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ORIG_ID                     827571 non-null  object 
 1   EMPRESA                     827571 non-null  object 
 2   DIVISION                    827571 non-null  object 
 3   REGION                      827571 non-null  object 
 4   HUB                         827570 non-null  object 
 5   PLAZA                       827571 non-null  object 
 6   ID_TRANS                    827571 non-null  object 
 7   NODO                        827571 non-null  object 
 8   RAMA                        827571 non-null  object 
 9   CUENTA                      827571 non-null  object 
 10  CALLE                       827571 non-null  object 
 11  NUMERO_INTERIOR             154831 non-null  object 
 12  NUMERO_EXTERIOR             827566 non-null  object 
 13  ENTRE_CALLES  

In [18]:
# Cruce con archivo CC con las fechas_filtradas
cuentas_filtradas = cruce[cruce['CUENTA'].isin(fechas_filtradas['Cliente'])]
cuentas_filtradas

,ORIG_ID,EMPRESA,DIVISION,REGION,HUB,PLAZA,ID_TRANS,NODO,RAMA,CUENTA,...,SIPRE_ID,FLG_HFC,FLG_GPON,FLG_FTTH_MTY,FLG_FTTH_AXTEL,TIPO_TECNOLOGIA,TIPO,LONGITUD,LATITUD,REG
75387,3,TVI,NORTE,NORESTE,MONTERREY,JUAREZ,JUA,N06,R9,22087597,...,18246576_0,Y,Y,N,N,GPON,NaN,-100.152363,25.644457,1.0
75642,3,TVI,NORTE,NORESTE,MONTERREY,JUAREZ,JUA,N06,R9,22095652,...,18246568_0,Y,Y,N,N,GPON,NaN,-100.15192,25.644319,1.0
75789,3,TVI,NORTE,NORESTE,MONTERREY,JUAREZ,JUA,N06,R9,22099928,...,18250755_0,Y,Y,N,N,GPON,NaN,-100.152568,25.644246,1.0
75902,3,TVI,NORTE,NORESTE,MONTERREY,JUAREZ,JUA,N06,R9,22104753,...,18247826_0,Y,Y,N,N,GPON,NaN,-100.154082,25.644885,1.0
76250,3,TVI,NORTE,NORESTE,MONTERREY,JUAREZ,JUA,N06,R9,22114537,...,18247815_0,Y,Y,N,N,GPON,NaN,-100.154691,25.645075,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684596,3,TVI,NORTE,NORESTE,MONTERREY,GUADALUPE,GPA,N01,RGP,41320685,...,911165957_2,N,Y,N,N,GPON,NaN,NaN,NaN,1.0
688832,3,TVI,NORTE,NORESTE,MONTERREY,GUADALUPE,GPA,N01,RGP,41460690,...,27337379_0,N,Y,N,N,GPON,NaN,-100.19379,25.687324,1.0
797428,3,TVI,NORTE,NORESTE,MONTERREY,GUADALUPE,GPA,N01,RGP,90105196,...,27337358_0,N,Y,N,N,GPON,NaN,-100.193790,25.687358,1.0
804784,3,TVI,NORTE,NORESTE,MONTERREY,GUADALUPE,GPA,N01,RGP,90145560,...,27337378_0,N,Y,N,N,GPON,REPEP,-100.193777,25.687324,1.0


In [19]:
len(cuentas_filtradas)

97

In [20]:
cuentas_filtradas.columns

Index(['ORIG_ID', 'EMPRESA', 'DIVISION', 'REGION', 'HUB', 'PLAZA', 'ID_TRANS',
       'NODO', 'RAMA', 'CUENTA', 'CALLE', 'NUMERO_INTERIOR', 'NUMERO_EXTERIOR',
       'ENTRE_CALLES', 'MANZANA', 'LOTE', 'COLONIA', 'CP',
       'MUNICIPIO_DELEGACION', 'CIUDAD', 'ESTADO',
       'TELEFONO_CELULAR_PRINCIPAL', 'TELEFONO_FIJO_PRINCIPAL', 'EMAIL',
       'NOMBRE_PRINCIPAL_CLI', 'APE_PATERNO', 'APE_MATERNO', 'PLAY', 'PAQUETE',
       'SEGMENTO', 'FACTURACION', 'FECHA_INSTALACION',
       'PLATAFORMA_TELCO_LEGACY', 'TIPO_ONT', 'MODELO_CAJA_VIDEO_CLI',
       'MODELO_EQ_INTERNET', 'SIPRE_ID', 'FLG_HFC', 'FLG_GPON', 'FLG_FTTH_MTY',
       'FLG_FTTH_AXTEL', 'TIPO_TECNOLOGIA', 'TIPO', 'LONGITUD', 'LATITUD',
       'REG'],
      dtype='object')

In [21]:
#Tomar solo los campos que deseamos
cuentas_filtradas =cuentas_filtradas[['CUENTA','NOMBRE_PRINCIPAL_CLI','APE_PATERNO','EMAIL']] 

In [22]:
#reseteo de index
archivo_gpon = cuentas_filtradas.reset_index(drop=True)

In [23]:
#renombres a columnas
archivo_gpon.rename(columns={'NOMBRE_PRINCIPAL_CLI':'NOMBRES'},inplace= True)

In [24]:
#Sección de limpieza de nombres
# Limpiar la columna 'NOMBRES'
if 'NOMBRES' in archivo_gpon.columns:
    archivo_gpon['NOMBRES'] = (
        archivo_gpon['NOMBRES']
        .str.strip()
        .replace(['', ' ', '  ', '   '], np.nan, regex=False)
        .replace(r'[^a-zA-Z\s]', np.nan, regex=True)
        .replace('.', '')
        .fillna('ESTIMADO')
        .str.upper()
        .str.split(" ").str.get(0)
        
    )
    # Remplazo de Marias 
    archivo_gpon['NOMBRES'] = archivo_gpon['NOMBRES'].replace('MA','MARIA')
    # Guardar en una matriz booleana aquellos nombres =< a 2 
    nombres_2_digitos = archivo_gpon['NOMBRES'].astype('str').str.len() <= 2
    # Hacer un replace con la matriz booleana usando el método .loc 
    archivo_gpon.loc[nombres_2_digitos, 'NOMBRES'] = 'ESTIMADO'

In [25]:
archivo_gpon

,CUENTA,NOMBRES,APE_PATERNO,EMAIL
0,22087597,ABIGAIL,CARRILLO,carrilloespinosamiguel@hotmail.com
1,22095652,JOB,MENDIETA,job.mendieta75@gmail.com
2,22099928,MARIA,SALDANA,m.ss@live.com.mx
3,22104753,VIRGINIA,ALVARADO,citlalidd8a@gmail.com
4,22114537,ZUU,SALAZAR,luis_cza15@hotmail.com
...,...,...,...,...
92,41320685,NAYDELI,MOLINA,molinanaydeli80@gmail.com
93,41460690,LUIS,ESQUIVEL,ldec_040892@hotmail.com
94,90105196,DIANA,SOLIS,nutridejesus87@gmail.com
95,90145560,GERARDO,RAMIREZ,geramirez123@hotmail.com


# Imputar sembrados

In [26]:
#Declaración de sembrados

# Crear un diccionario con los datos proporcionados
sembrados = {
    'CUENTA': ['MKD000001', 'MKD000002', 'MKD000003', 'MKD000004', 'MKD000005', 'MKD000006'],
    'NOMBRES': ['OILED', 'KARLA', 'LILIANA', 'ROBERTO', 'CLAUDIA', 'LEONARDO'],
    'APE_PATERNO': ['SAENZ', 'PUENTES', 'GOMEZ', 'JASSO', 'ARAUJO', 'GONZALEZ'],
    'EMAIL': ['osaenzh@gmail.com', 'puenteskarla302@gmail.com', 'lgomez@izzi.mx', 
              'roberto.jassop@gmail.com', 'caraujo@izzi.mx', 'lgonzalezc@izzi.mx']
}

In [27]:
# Crear el DataFrame
df_sembrados = pd.DataFrame(sembrados)

# Mostrar el DataFrame
print(df_sembrados)

      CUENTA   NOMBRES APE_PATERNO                      EMAIL
0  MKD000001     OILED       SAENZ          osaenzh@gmail.com
1  MKD000002     KARLA     PUENTES  puenteskarla302@gmail.com
2  MKD000003   LILIANA       GOMEZ             lgomez@izzi.mx
3  MKD000004   ROBERTO       JASSO   roberto.jassop@gmail.com
4  MKD000005   CLAUDIA      ARAUJO            caraujo@izzi.mx
5  MKD000006  LEONARDO    GONZALEZ         lgonzalezc@izzi.mx


In [28]:
# Concatenar df_sembrados a archivo_gpon
result = pd.concat([archivo_gpon, df_sembrados], axis=0)

In [29]:
#Conteo final
result.dropna(inplace=True)
len(result)

103